In [2]:
# Vizualizing the data from storm sweeps

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys
import pickle
from multiprocessing import Pool

import seaborn as sns

from weis.aeroelasticse import FileTools

sys.path.append(os.path.abspath("/Users/mchetan/Desktop/nrel/pyDatView/pydatview/tools"))

from fatigue import *

In [11]:
# Locations of the folders
tc_steady = '../eagleData/31JAN24/steady/'
tc_unsteady = '../eagleData/31JAN24/unsteady/'

tc_fast_steady = '../eagleData/31JAN24-fast/steady/'
tc_fast_unsteady = '../eagleData/31JAN24-fast/unsteady/'

of351_steady = '../eagleData/31JAN24-of-351/steady/'
of351_unsteady = '../eagleData/31JAN24-of-351/unsteady/'

cm_dict = FileTools.load_yaml('../eagleData/31JAN24/steady/case_matrix.yaml', package=1)

cm_df = pd.DataFrame(cm_dict)

In [ ]:
# listing the yaw angles
# cm_df.keys()
np.array(cm_df[cm_df.columns[10]]).astype(float).round(0).astype(int)

In [6]:
def process_Stats(openfastDF, fileName= None, startTime = 420.0):
    # Passed the OpenFAST dataframe, returns a dataframe with the stats for all the channels

    statsDF = pd.DataFrame(index = openfastDF.columns,  columns=['mean', 'std', 'max', 'min', 'DEL5', 'DEL10', 'DEL12'])

    success = True

    # Trim the data to start at startTime, if empty, write to error file and use whole data
    temp = openfastDF[openfastDF[('Time', '(s)')] >= startTime]

    if temp.empty:
        print(f'No data after {startTime} seconds')
        
        with open('error.txt', 'a') as f:
            f.write(f"{fileName} has no data after {openfastDF[('Time', '(s)')][len(openfastDF[('Time', '(s)')])-1]} seconds\n")

        temp = openfastDF

        success = False

    for col in temp.columns:

        # print(f'working on {col}')

        if col == ('Time', '(s)'):
            continue

        statsDF.loc[col] = [temp[col].mean(), temp[col].std(), temp[col].max(), temp[col].min(),
                            equivalent_load(temp[('Time', '(s)')], temp[col], m=5), equivalent_load(temp[('Time', '(s)')], temp[col], m=10), equivalent_load(temp[('Time', '(s)')], temp[col], m=12)]


    return statsDF, success

In [60]:
# Loading a single pickel file for testing
df = pd.read_pickle('eagleData/23JAN24/steady/storm-test_08.out.pkl')

# dump to csv with tab delimiter
df.to_csv('eagleData/23JAN24/steady/storm-test_08.out.csv', sep='\t', index=False)


In [12]:
# for file in os.listdir(tc_steady):
#     if file.endswith('.out'):
#         df = pd.read_csv(tc_steady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
#         # df = pd.read_pickle(tc_steady + file)
#         df.to_csv(tc_steady + file.replace('.out', '.csv'), sep='\t', index=False)

# for file in os.listdir(tc_unsteady):
#     if file.endswith('.out'):
#         df = pd.read_csv(tc_unsteady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
#         # df = pd.read_pickle(tc_unsteady + file)
#         df.to_csv(tc_unsteady + file.replace('.out', '.csv'), sep='\t', index=False)

for file in os.listdir(tc_fast_steady):
    if file.endswith('.out'):
        df = pd.read_csv(tc_fast_steady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
        # df = pd.read_pickle(tc_fast_steady + file)
        df.to_csv(tc_fast_steady + file.replace('.out', '.csv'), sep='\t', index=False)

for file in os.listdir(tc_fast_unsteady):
    if file.endswith('.out'):
        df = pd.read_csv(tc_fast_unsteady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
        # df = pd.read_pickle(tc_fast_unsteady + file)
        df.to_csv(tc_fast_unsteady + file.replace('.out', '.csv'), sep='\t', index=False)

for file in os.listdir(of351_steady):
    if file.endswith('.out'):
        df = pd.read_csv(of351_steady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
        # df = pd.read_pickle(of351_steady + file)
        df.to_csv(of351_steady + file.replace('.out', '.csv'), sep='\t', index=False)

for file in os.listdir(of351_unsteady):
    if file.endswith('.out'):
        df = pd.read_csv(of351_unsteady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
        # df = pd.read_pickle(of351_unsteady + file)
        df.to_csv(of351_unsteady + file.replace('.out', '.csv'), sep='\t', index=False)

/var/folders/6r/0bmv_2bd2fx38vj0q38h019cv01fld/T/ipykernel_75783/2649273188.py:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(tc_fast_steady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
/var/folders/6r/0bmv_2bd2fx38vj0q38h019cv01fld/T/ipykernel_75783/2649273188.py:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(tc_fast_steady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
/var/folders/6r/0bmv_2bd2fx38vj0q38h019cv01fld/T/ipykernel_75783/2649273188.py:15: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(tc_fast_steady+"/"+file,skiprows=6,header=[0,1],dtype=np.float32,delim_whitespace=True)
/var

In [ ]:
df[('Time', '(s)')][len(df[('Time', '(s)')]) - 1]

In [18]:
df_test = pd.read_csv('/Users/mchetan/Desktop/nrel/storm/eagleData/31JAN24/steady/storm-test_00.csv',sep='\t', header=[0,1])
df_test

,Time,ConvIter,ConvError,NumUJac,Wind1VelX,Wind1VelY,Wind1VelZ,Wind2VelX,Wind2VelY,Wind2VelZ,...,B1N2Cl,B1N3Cl,B1N4Cl,B1N5Cl,B1N6Cl,B1N7Cl,B1N8Cl,B1N9Cl,GenPwr,GenTq
,(s),(-),(-),(-),(m/s),(m/s),(m/s),(m/s),(m/s),(m/s),...,(-),(-),(-),(-),(-),(-),(-),(-),(kW),(kN-m)
0,0.00,5.0,0.000034,0.0,4.070,0.000217,0.000045,4.070,0.000217,0.000045,...,-0.2650,-0.1337,-0.05986,-0.02148,0.01157,0.03568,0.06848,0.07373,0.000000e+00,-0.1
1,0.01,2.0,0.000011,0.0,4.071,0.000217,0.000045,4.071,0.000217,0.000045,...,-0.2656,-0.1338,-0.05997,-0.02160,0.01173,0.03576,0.06859,0.07420,1.635000e-07,-0.1
2,0.02,2.0,0.000048,0.0,4.073,0.000217,0.000045,4.073,0.000217,0.000045,...,-0.2676,-0.1352,-0.06121,-0.02274,0.01192,0.03726,0.07059,0.07744,-3.394000e-06,-0.1
3,0.03,2.0,0.000041,0.0,4.074,0.000217,0.000045,4.074,0.000217,0.000045,...,-0.2693,-0.1385,-0.06197,-0.02279,0.01124,0.03890,0.07318,0.07977,-5.680000e-06,-0.1
4,0.04,2.0,0.000014,0.0,4.075,0.000217,0.000045,4.075,0.000217,0.000045,...,-0.2692,-0.1425,-0.06186,-0.02084,0.01055,0.03892,0.07466,0.07853,-5.129000e-06,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111107,1111.07,6.0,0.000019,0.0,23.960,-0.486100,-1.237000,32.480,0.404500,0.016350,...,-0.4017,-0.3311,-0.45470,-0.62300,-0.81690,-0.73290,-0.68880,-0.69340,-4.098000e-03,-0.1
111108,1111.08,6.0,0.000016,0.0,23.910,-0.632400,-1.179000,32.550,0.399600,-0.009377,...,-0.3817,-0.2948,-0.39760,-0.55990,-0.77970,-0.75000,-0.71720,-0.73640,-4.618000e-03,-0.1
111109,1111.09,5.0,0.000033,0.0,23.870,-0.778700,-1.120000,32.610,0.394700,-0.035110,...,-0.3594,-0.2512,-0.32370,-0.46620,-0.67500,-0.76960,-0.74160,-0.64300,-5.123000e-03,-0.1


In [19]:
# process and save all the stats for all the folders

runFail = {'success':None, 'error':None, 'notes':None, 'author':'Chetan'}


# tc_steady
for file in os.listdir(tc_steady):
    if file.endswith('.csv'):
        print(f'Processing {file[:-8]} in tc_steady')
        df = pd.read_csv(tc_steady + file, sep='\t', header=[0,1])
        stats, success = process_Stats(df)
        runFail['success'] = success
        with open(tc_steady + file.replace('.csv', '_runStatus.pkl'), 'wb') as f: 
            pickle.dump(runFail, f)
        stats.to_csv(tc_steady + file.replace('.csv', '_stats.csv'), sep='\t')

# tc_unsteady
for file in os.listdir(tc_unsteady):
    if file.endswith('.csv'):
        print(f'Processing {file[:-8]} in tc_unsteady')
        df = pd.read_csv(tc_unsteady + file, sep='\t', header=[0,1])
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(tc_unsteady + file.replace('.csv', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_csv(tc_unsteady + file.replace('.csv', '_stats.csv'), sep='\t')

# tc_fast_steady
for file in os.listdir(tc_fast_steady):
    if file.endswith('.csv'):
        print(f'Processing {file[:-8]} in tc_fast_steady')
        df = pd.read_csv(tc_fast_steady + file, sep='\t', header=[0,1])
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(tc_fast_steady + file.replace('.csv', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_csv(tc_fast_steady + file.replace('.csv', '_stats.csv'), sep='\t')

# tc_fast_unsteady
for file in os.listdir(tc_fast_unsteady):
    if file.endswith('.csv'):
        print(f'Processing {file[:-8]} in tc_fast_unsteady')
        df = pd.read_csv(tc_fast_unsteady + file, sep='\t', header=[0,1])
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(tc_fast_unsteady + file.replace('.csv', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_csv(tc_fast_unsteady + file.replace('.csv', '_stats.csv'), sep='\t')

# of351_steady
for file in os.listdir(of351_steady):
    if file.endswith('.csv'):
        print(f'Processing {file[:-8]} in of351_steady')
        df = pd.read_csv(of351_steady + file, sep='\t', header=[0,1])
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(of351_steady + file.replace('.csv', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_csv(of351_steady + file.replace('.csv', '_stats.csv'), sep='\t')

# of351_unsteady
for file in os.listdir(of351_unsteady):
    if file.endswith('.csv'):
        print(f'Processing {file[:-8]} in of351_unsteady')
        df = pd.read_csv(of351_unsteady + file, sep='\t', header=[0,1])
        stats, success = process_Stats(df)         
        runFail['success'] = success         
        with open(of351_unsteady + file.replace('.csv', '_runStatus.pkl'), 'wb') as f:              
            pickle.dump(runFail, f)
        stats.to_csv(of351_unsteady + file.replace('.csv', '_stats.csv'), sep='\t')

Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing storm-tes in tc_steady
Processing sto

In [ ]:

df1 = pd.read_pickle('eagleData/23JAN24/steady/storm-test_00.out.pkl')
stats, success = process_Stats(df1)         
stats.attribute = success
print(stats.attribute)

with open('testPickly.pkl', 'wb') as f:
    pickle.dump(stats, f)

with open('testPickly.pkl', 'rb') as f:
    df = pickle.load(f)

print(df.attribute)

df.attrs


In [48]:
# function to Create a Dataframe of the stats of all runs and plot it as a heatmap, NOT EFFICIENT BUT WORKS!

def pltstats(ChanName, statType):

    # dict of the paths
    runPaths = {'tc_steady': tc_steady, 'tc_unsteady': tc_unsteady, 'tc_fast_steady': tc_fast_steady, 'tc_fast_unsteady': tc_fast_unsteady, 'of351_steady': of351_steady, 'of351_unsteady': of351_unsteady}

    plotDF = pd.DataFrame(columns=runPaths.keys())

    # success matrix
    success = pd.DataFrame(columns=runPaths.keys())

    # looping through all the stat files then runt ypes


    # looping through cm_df rows
    for index, row in cm_df.iterrows():

        # looping through the run types dict
        for runType, path in runPaths.items():

            # building the file name
            fileName = f'{path}/storm-test_{index:02d}_stats.pkl'

            # reading the file
            # print(f'Processing {fileName}')
            df = pd.read_pickle(fileName)

            # adding the value to the plotDF
            plotDF.loc[index, runType] = df.loc[ChanName, statType]

            # adding the success flag to the success matrix
            with open(fileName.replace('_stats.pkl', '_runStatus.pkl'), 'rb') as f:
                runFail = pickle.load(f)
                success.loc[index, runType] = runFail['success']

    # plotting the heatmap
    plt.figure(figsize=(20, 20))
    ax = sns.heatmap(plotDF.apply(pd.to_numeric), annot=True, fmt=".2f", cmap='viridis')

    # # set failed runs to red text color
    # for i in range(success.shape[0]):
    #     for j in range(success.shape[1]):
    #         # print(f'failed run at {i}, {j} = {ax.texts[i*success.shape[0] + j]}')
    #         if success.iloc[i, j] == False:
    #             # setting the color of the cell text to red
                
    #             if ax.texts[i*success.shape[1] + j].get_text() != 'nan':
    #                 ax.texts[i*success.shape[1] + j].set_color('red')


    ax.set_title(f'{ChanName} {statType} for all runs')

    ax.xaxis.tick_top()

    # replace x ticks with yaw angles
    ax.set_yticks(ticks=np.array(cm_df.index), labels=np.array(cm_df[cm_df.columns[10]]).astype(float).round(0).astype(int))



    # Go through 

    ax.set_xlabel('Run Type')
    ax.set_ylabel('Yaw Angle [deg]')

    # plt.show()

    # Save the plot image in a new folder
    plotDF.to_pickle(f'HeatMaps/data/{ChanName}_{statType}.pkl') 

    plt.savefig(f'HeatMaps/{ChanName}_{statType}.png')

    plt.close()

    return success



In [ ]:
df = pd.read_pickle('eagleData/23JAN24/steady/storm-test_00.out.pkl')
canList = df.columns
list(canList)

In [49]:
InterstingChans = [
    ('TTDspFA', '(m)'),
    ('TTDspSS', '(m)'),
    ('TwrBsMxt', '(kN-m)'),
    ('TwrBsMyt', '(kN-m)'),
    
    ('B1RootMxr', '(N-m)'),
    ('B1RootMyr', '(N-m)'),
    ('B1TipTDxr', '(m)'),
    ('B1TipTDyr', '(m)'),

    ('B2RootMxr', '(N-m)'),
    ('B2RootMyr', '(N-m)'),
    ('B2TipTDxr', '(m)'),
    ('B2TipTDyr', '(m)'),

    ('B3RootMxr', '(N-m)'),
    ('B3RootMyr', '(N-m)'),
    ('B3TipTDxr', '(m)'),
    ('B3TipTDyr', '(m)'),

    ('B1N1Cl', '(-)'),
    ('B1N2Cl', '(-)'),
    ('B1N3Cl', '(-)'),
    ('B1N4Cl', '(-)'),
    ('B1N5Cl', '(-)'),
    ('B1N6Cl', '(-)'),
    ('B1N7Cl', '(-)'),
    ('B1N8Cl', '(-)'),
    ('B1N9Cl', '(-)'),
]

# Single test run
status = pltstats(('B1N9Cl', '(-)'), 'mean')

# for chan in InterstingChans:
#     pltstats(chan, 'mean')
#     pltstats(chan, 'std')
#     pltstats(chan, 'DEL5')
#     pltstats(chan, 'DEL10')
#     pltstats(chan, 'DEL12')

In [54]:
status

,tc_steady,tc_unsteady,tc_fast_steady,tc_fast_unsteady,of351_steady,of351_unsteady
0,True,False,False,False,True,False
1,True,False,True,False,True,False
2,True,False,True,False,True,False
3,True,False,True,False,True,False
4,True,False,True,False,True,False
5,True,False,True,False,True,False
6,True,False,True,False,True,False
7,True,False,True,False,True,False
8,True,False,True,False,False,False
9,True,True,True,True,True,False
